# EazyML Image XAI: Brain MRI Data

In [ ]:
!pip install --upgrade eazyml-xai-image
!pip install gdown python-dotenv

## Define imports

In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import gdown
import tensorflow as tf
import segmentation_models as sm
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from eazyml_xai_image import (
    ez_init,
    ez_xai_image_explain
)

from dotenv import load_dotenv
load_dotenv()

### Initialize EazyML
The `ez_init` function uses the `EAZYML_ACCESS_KEY` environment variable for authentication. If the variable is not set, it defaults to a trial license.

In [ ]:
ez_init(access_key=os.getenv('EAZYML_ACCESS_KEY'))

## Download image data and model

In [ ]:
# download data
gdown.download_folder(id='10BRxPpPlscUUIUEmKyQgAZenVzEoSrdk')
# download model
gdown.download_folder(id='16kcSmVus6gQvjhTTvaT3EVlGeyov-71q')

## Load the model, Get the outputs, and Save them

In [ ]:
model_path = "model/unet_brain_mri_seg.hdf5"

def preprocess_image(img):
    image_size = 256
    return cv2.resize(img, (image_size,image_size))/255

In [ ]:
model = tf.keras.models.load_model(model_path, compile=False)

for j in range(11, 18):
    filename = f"data/TCGA_CS_4941_19960909_{j}.tif"
    predicted_filename = f"data/kaggle_3m_test_{j}.csv"
    pred = model.predict(np.expand_dims(preprocess_image(cv2.imread(filename)), 0))[0,:,:,0]
    pred = np.where(pred > 0.2, pred, 0)
    cv2.imwrite(f"data/kaggle_3m_test_op_{j}.jpg", cv2.imread(filename))
    np.savetxt(predicted_filename, pred, delimiter=",")
    cv2.imwrite(predicted_filename.replace(".csv", ".jpg"), cv2.imread(filename))

In [ ]:
filename = "data/kaggle_3m_test_11.jpg"
model_path = "model/unet_brain_mri_seg.hdf5"
predicted_filename = "data/kaggle_3m_test_11.csv"

In [ ]:
j = 11
filename = f"data/TCGA_CS_4941_19960909_{j}.tif"
predicted_filename = f"data/kaggle_3m_test_{j}.csv"
pred = model.predict(np.expand_dims(cv2.resize(cv2.imread(filename), (256, 256))/255, 0))
np.sum(np.where(np.array(pred) < 0.8, 1, 0))

In [ ]:
plt.subplots(1, 2, figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.imread(filename))
plt.subplot(1, 2, 2)
plt.imshow(np.array(pd.read_csv(predicted_filename)))

# Confidence Score and Explanation

### Defining any Preprocessing functions

In [ ]:
def preprocess_image(img):
    image_size = 256
    return tf.image.resize(img, (image_size,image_size))/255

required_functions = {
    "input_preprocess_fn": preprocess_image
}

### Default options

In [ ]:
ez_xai_image_explain(filename, model_path, predicted_filename, options = {})

## Confidence Score

### Weighted Moments

In [ ]:
ez_xai_image_explain(filename, model_path, predicted_filename, {"score_strategy": "weighted-moments"})

### Entropy

In [ ]:
ez_xai_image_explain(filename, model_path, predicted_filename, {"score_strategy": "log-info"})

### Lime Confidence

In [ ]:
ez_xai_image_explain(filename, model_path, predicted_filename, {"score_strategy": "lime-confidence"})

## Image Explanation

### Gradcam

In [ ]:
options = {
    "xai_strategy": "gradcam",
    "gradcam_layer": "conv2d_36",
    "required_functions": required_functions
}
resp = ez_xai_image_explain(filename, model_path, predicted_filename, options)
print(resp)
plt.figure(figsize=(12, 12))
plt.imshow(cv2.imread(resp['explanations']['explanation'][0]))

### Highres Gradcam

In [ ]:
options = {
    "xai_strategy": "highres-gradcam",
    "required_functions": required_functions
}
resp = ez_xai_image_explain(filename, model_path, predicted_filename, options)
print(resp)
plt.figure(figsize=(12, 12))
plt.imshow(cv2.imread(resp['explanations']['explanation'][0]))

### Image Lime

In [ ]:
options = {
    "xai_strategy": "image-lime",
    "required_functions": required_functions
}
resp = ez_xai_image_explain(filename, model_path, predicted_filename, options)
print(resp)
plt.figure(figsize=(12, 12))
plt.imshow(cv2.imread(resp['explanations']['explanation'][0]))

# Active Learning

In [ ]:
filenames = [
    "data/kaggle_3m_test_11.jpg",
    "data/kaggle_3m_test_12.jpg",
    "data/kaggle_3m_test_13.jpg",
    "data/kaggle_3m_test_14.jpg",
    "data/kaggle_3m_test_15.jpg",
]
model_path = "model/unet_brain_mri_seg.hdf5"
predicted_filenames = [
    "data/kaggle_3m_test_11.csv",
    "data/kaggle_3m_test_12.csv",
    "data/kaggle_3m_test_13.csv",
    "data/kaggle_3m_test_14.csv",
    "data/kaggle_3m_test_15.csv",
]

In [ ]:
from eazyml_xai_image import ez_image_active_learning

### Default

In [ ]:
ez_image_active_learning(filenames, model_path, predicted_filenames)

### Weighted Moments

In [ ]:
ez_image_active_learning(filenames, model_path, predicted_filenames, {"score_strategy": "weighted-moments"})

### Entropy

In [ ]:
ez_image_active_learning(filenames, model_path, predicted_filenames, {"score_strategy": "log-info"})

### Lime Confidence

In [ ]:
# ez_image_active_learning(filenames, model_path, predicted_filenames, {"score_strategy": "lime-confidence"})

# Online Learning

### Preparing data for training and validation data

In [ ]:
# Training data
tr_data = {
    "inputs": [
        "data/kaggle_3m_test_11.jpg",
        "data/kaggle_3m_test_12.jpg",
        "data/kaggle_3m_test_13.jpg",
        "data/kaggle_3m_test_14.jpg",
        "data/kaggle_3m_test_15.jpg",
    ],
    "labels": [
        "data/kaggle_3m_test_op_11.jpg",
        "data/kaggle_3m_test_op_12.jpg",
        "data/kaggle_3m_test_op_13.jpg",
        "data/kaggle_3m_test_op_14.jpg",
        "data/kaggle_3m_test_op_15.jpg",
    ]
}

# Model path
model_path = "model/unet_brain_mri_seg.hdf5"

# Validation data
val_data = {
    "inputs": [
        "data/kaggle_3m_test_11.jpg",
        "data/kaggle_3m_test_12.jpg",
    ],
    "labels": [
        "data/kaggle_3m_test_op_11.jpg",
        "data/kaggle_3m_test_op_12.jpg",
    ]
}

In [ ]:
from eazyml_xai_image import ez_image_model_evaluate, ez_image_online_learning

In [ ]:
training_data_path = f"./train_data.csv"
# model_path = f"model_to_be_trained.h5"
new_model_path = "trained_model.h5"
validation_data_path = f"val_data.csv"
fields = ["inputs", "labels"]

# Loss function for the image segmentation model
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# Preprocessing function for inputs
def preprocess_image(x):
    x = tf.image.resize(x, (256, 256))
    pre_ = sm.get_preprocessing("efficientnetb6") 
    return pre_(x)

# Preprocessing function for masks
def preprocess_label(x):
    mask = tf.cast(tf.where(x == 1, 1., 0.), dtype=tf.float32)
    mask = tf.image.resize(mask, (256, 256))
    return mask

req_fns = {
    "input_preprocess_fn": preprocess_image,
    "label_preprocess_fn": preprocess_label,
    "loss_fn": total_loss,
    "metric_fns": {
        "metric_iou": sm.metrics.IOUScore(threshold=0.5),
        "metric_fscore":sm.metrics.FScore(threshold=0.5)
    }
}

def get_eval_options():
    # Options to call ez_image_model_evaluate
    eval_options = {}
    eval_options["batch_size"] = 1
    eval_options["required_functions"] = req_fns
    eval_options["required_functions"]["path_to_save"] = "./pre_fns"
    return eval_options

In [ ]:
tr_options = {}
tr_options["training_parameters"] = {
    "batchsize" : 1,
    "epochs" : 2,
    "learning_rate" : 1e-4,
}
tr_options["ol_strategy"] = "fine-tuning"
tr_options["tr_strategy"] = "normal"
tr_options["validation_data_path"] = validation_data_path
tr_options["new_model_path"] = new_model_path
tr_options["required_functions"] = req_fns
tr_options["log_file"] = f"suture_online_log.csv"

In [ ]:
list_dict = {fields[0]: val_data["inputs"], fields[1]: val_data["labels"]}
df = pd.DataFrame(list_dict)
df.to_csv(validation_data_path)

### Evaluate the current model on Validation Data

In [ ]:
response = ez_image_model_evaluate(validation_data_path,
                                   model_path,
                                   get_eval_options())
prev_val = response['eval_info']
print("Initial weights evaluated on validation data.")
print("Loss: ", prev_val[0], ", IOU Score: ", prev_val[1], ", F-Score: ", prev_val[2], "\n")

In [ ]:
# Training Data
list_dict = {fields[0]: tr_data["inputs"], fields[1]: tr_data["labels"]}
df = pd.DataFrame(list_dict)
df.to_csv(training_data_path)

# Validation Data
list_dict = {fields[0]: val_data["inputs"], fields[1]: val_data["labels"]}
df = pd.DataFrame(list_dict)
df.to_csv(validation_data_path)

### Training the model on the training data

In [ ]:
history = ez_image_online_learning(training_data_path,
                        model_path,
                        tr_options)
print("Retraining completed.")
print(history, "\n")

### Saving the new model as a h5 file

In [ ]:
model = tf.keras.models.load_model(model_path, compile=False)
model.load_weights(new_model_path)
model.save(new_model_path)

### Evaluating the new model on the validation model

In [ ]:
response = ez_image_model_evaluate(validation_data_path,
                                   new_model_path,
                                   get_eval_options())
prev_val = response['eval_info']
print("Initial weights evaluated on validation data.")
print("Loss: ", prev_val[0], ", IOU Score: ", prev_val[1], ", F-Score: ", prev_val[2], "\n")